In [7]:
import joblib
import pandas as pd
import numpy as np

In [27]:
# --- Load Data ---
df_demog = pd.read_excel('Data/DataScientist_CaseStudy_Dataset.xlsx', sheet_name='Soc_Dem')
df_bal = pd.read_excel('Data/DataScientist_CaseStudy_Dataset.xlsx', sheet_name='Products_ActBalance')
df_in_out = pd.read_excel('Data/DataScientist_CaseStudy_Dataset.xlsx', sheet_name='Inflow_Outflow')
df_target = pd.read_excel('Data/DataScientist_CaseStudy_Dataset.xlsx', sheet_name='Sales_Revenues')
df_desc = pd.read_excel('Data/DataScientist_CaseStudy_Dataset.xlsx', sheet_name='Description')

# Data merge pipeline
df_master = (df_demog
             .merge(df_in_out, on='Client')
             .merge(df_bal, on='Client'))

df_master_market = df_master[~df_master['Client'].isin(client for client in df_target['Client'])].fillna(0)

# Adding feature engineering
df_master_market['Avg_CA_bal'] = df_master_market['ActBal_CA']/df_master_market['Count_CA']
df_master_market['Avg_SA_bal'] = df_master_market['ActBal_SA']/df_master_market['Count_SA']
df_master_market['Avg_MF_bal'] = df_master_market['ActBal_MF']/df_master_market['Count_MF']
df_master_market['utilization_CA'] = np.minimum(100,df_master_market['VolumeDeb_CA']/df_master_market['ActBal_CA'])

df_master_market = df_master_market.drop(columns=['Client'])

In [30]:
df_market_final = df_master_market.copy()
products = ["CL", "CC", "MF"]

for product in products:

    # Predict probabilities
    model = joblib.load(f"Model/Sale_{product}_model.joblib")
    binning_process = joblib.load(f"Output/Sale_{product}_binning_process.joblib")
    selected_features = joblib.load(f"Output/Sale_{product}_selected_features.joblib")

    # --- Load or Prepare New Data for Scoring ---
    X_new_woe = binning_process.transform(df_market_final, metric="woe")
    X_new_final = X_new_woe[selected_features]
    pred_proba = model.predict_proba(X_new_final)[:, 1]

    df_market_final[f"prob_{product}"] = pred_proba

    # Predict Revenue
    # Load the model from file

    df_master_market['Sex'] = df_master_market['Sex'].astype('category')
    regressor_loaded = joblib.load(f"Model/Revenue_{product}_model.joblib")
    # Predict
    y_pred = regressor_loaded.predict(df_master_market)
    df_market_final[f"rev_{product}"] = y_pred

    df_market_final[f"expected_revenue_{product}"] = df_market_final[f"prob_{product}"] * df_market_final[f"rev_{product}"]


C:\Users\joeytian\anaconda3\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)
C:\Users\joeytian\anaconda3\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)
C:\Users\joeytian\anaconda3\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: divide by zero encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


In [31]:
# Select the expected revenue columns
expected_revenue_cols = [f"expected_revenue_{p}" for p in products]

# Max expected revenue across products
df_market_final["expected_revenue"] = df_market_final[expected_revenue_cols].max(axis=1)

# Product with highest expected revenue (argmax)
df_market_final["selected_product"] = df_market_final[expected_revenue_cols].idxmax(axis=1).str.replace("expected_revenue_", "")

In [36]:
df_top100 = df_market_final.sort_values(by="expected_revenue", ascending=False).head(100)
df_top100.to_csv("Client_List_Final.csv")

In [37]:
df_top100

,Sex,Age,Tenure,VolumeCred,VolumeCred_CA,TransactionsCred,TransactionsCred_CA,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,...,rev_CL,expected_revenue_CL,prob_CC,rev_CC,expected_revenue_CC,prob_MF,rev_MF,expected_revenue_MF,expected_revenue,selected_product
1267,M,20,6,9618.215000,9605.281071,11,8,16060.417143,10703.274286,9714.285714,...,12.224891,11.769878,0.172722,8.108205,1.400467,0.029075,7.791911,0.226548,11.769878,CL
1389,F,58,200,4372.773929,2964.290357,23,14,5868.460714,4031.925000,157.142857,...,12.295677,11.263614,0.030042,8.088335,0.242993,0.056675,7.912549,0.448440,11.263614,CL
1119,M,27,216,899.314643,857.432857,8,6,886.323571,844.463571,392.857143,...,12.387659,11.171847,0.205148,8.120225,1.665844,0.027459,7.791911,0.213959,11.171847,CL
695,M,20,55,1099.482500,1099.482500,2,2,5698.346429,5698.346429,2892.857143,...,12.224891,10.888098,0.162783,8.108205,1.319874,0.200054,7.791911,1.558803,10.888098,CL
1200,M,9,179,14587.096429,14092.655714,24,20,2603.055357,2109.550000,571.428571,...,12.469619,10.660398,0.136382,8.088335,1.103101,0.161668,7.791911,1.259706,10.660398,CL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244,F,44,175,1521.250714,1459.827143,5,4,1591.185714,1584.507143,1035.714286,...,12.283386,6.919271,0.126544,8.108205,1.026043,0.114501,7.783803,0.891256,6.919271,CL
873,F,51,137,1124.046071,1124.046071,5,5,1104.178571,1104.178571,0.000000,...,12.247576,6.908813,0.146423,8.125657,1.189783,0.378273,7.783803,2.944404,6.908813,CL
756,F,3,15,67.498571,67.455357,5,4,471.878929,404.425357,107.142857,...,12.469619,6.903665,0.230107,8.103205,1.864605,0.122996,7.791911,0.958376,6.903665,CL
1226,F,73,219,634.995000,633.544286,6,3,910.214286,908.785714,607.142857,...,12.315372,6.870734,0.469372,8.153753,3.827143,0.171096,7.847497,1.342677,6.870734,CL
